In [1]:
# %pip install setfit[optuna]==0.3.0 datasets -U

     ---------------------------------------- 0.0/486.2 kB ? eta -:--:--
     -- ------------------------------------ 30.7/486.2 kB 1.3 MB/s eta 0:00:01
     --------- ---------------------------- 122.9/486.2 kB 1.2 MB/s eta 0:00:01
     ----------------- -------------------- 225.3/486.2 kB 1.5 MB/s eta 0:00:01
     -------------------------------- ----- 409.6/486.2 kB 2.1 MB/s eta 0:00:01
     -------------------------------------- 486.2/486.2 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/362.3 kB ? eta -:--:--
     ------------------------- ----------- 245.8/362.3 kB 15.7 MB/s eta 0:00:01
     -------------------------------------- 362.3/362.3 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -----------------------------------

In [1]:
from datasets import load_dataset,concatenate_datasets

# Load the dataset
dataset = load_dataset("ag_news")

# create train dataset
seed=20
labels = 4
samples_per_label = 8
sampled_datasets = []
# find the number of samples per label
for i in range(labels):
    sampled_datasets.append(dataset["train"].filter(lambda x: x["label"] == i).shuffle(seed=seed).select(range(samples_per_label)))

# concatenate the sampled datasets
train_dataset = concatenate_datasets(sampled_datasets)

# create test dataset
test_dataset = dataset["test"]

Using custom data configuration default


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to C:\Users\DongViet\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function <lambda> at 0x0000018656754D30> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/120 [00:00<?, ?ba/s]

In [2]:
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss

# Load a SetFit model from Hub
model_id = "sentence-transformers/all-mpnet-base-v2"
model = SetFitModel.from_pretrained(model_id)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=1,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for constrastive learning
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate()

print(f"model used: {model_id}")
print(f"train dataset: {len(train_dataset)} samples")
print(f"accuracy: {metrics['accuracy']}")

C:\Users\DongViet\anaconda3\envs\deepwork\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DongViet\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 1280
  Num epochs = 1
  Total optimization steps = 1280
  Total train batch size = 1


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1280 [00:00<?, ?it/s]

***** Running evaluation *****


model used: sentence-transformers/all-mpnet-base-v2
train dataset: 32 samples
accuracy: 0.8631578947368421


In [3]:
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss

# model specfic hyperparameters
def model_init(params):
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    model_id = params.get("model_id", "sentence-transformers/all-mpnet-base-v2")
    model_params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained(model_id, **model_params)

# training hyperparameters
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 5),
        "batch_size": trial.suggest_categorical("batch_size", [1, 2, 4, 8]),
        "num_iterations": trial.suggest_categorical("num_iterations", [5, 10, 20, 40, 80]),
        "seed": trial.suggest_int("seed", 1, 40),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
        "model_id": trial.suggest_categorical(
            "model_id",
            [
                "sentence-transformers/all-mpnet-base-v2",
                "sentence-transformers/all-MiniLM-L12-v1",
            ],
        ),
    }


trainer = SetFitTrainer(
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
)

best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=100)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2023-06-27 12:58:47,855] A new study created in memory with name: no-name-a9b852a5-81a4-4427-8d0e-c839ad7425ed
Trial: {'learning_rate': 4.66916130572777e-06, 'num_epochs': 4, 'batch_size': 2, 'num_iterations': 5, 'seed': 4, 'max_iter': 239, 'solver': 'newton-cg', 'model_id': 'sentence-transformers/all-mpnet-base-v2'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 320
  Num epochs = 4
  Total optimization steps = 160
  Total train batch size = 2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/160 [00:00<?, ?it/s]

Iteration:   0%|          | 0/160 [00:00<?, ?it/s]

Iteration:   0%|          | 0/160 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2560 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2560 [00:00<?, ?it/s]

[W 2023-06-27 13:15:36,355] Trial 10 failed with parameters: {'learning_rate': 6.91436936940602e-06, 'num_epochs': 5, 'batch_size': 2, 'num_iterations': 80, 'seed': 40, 'max_iter': 62, 'solver': 'newton-cg', 'model_id': 'sentence-transformers/all-mpnet-base-v2'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\DongViet\anaconda3\envs\deepwork\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\DongViet\anaconda3\envs\deepwork\lib\site-packages\setfit\integrations.py", line 27, in _objective
    trainer.train(trial=trial)
  File "C:\Users\DongViet\anaconda3\envs\deepwork\lib\site-packages\setfit\trainer.py", line 276, in train
    self.model.model_body.fit(
  File "C:\Users\DongViet\anaconda3\envs\deepwork\lib\site-packages\sentence_transformers\SentenceTransformer.py", line 721, in fit
    loss_value = loss_model(features, labels)
  File "C:\Users\DongViet\anacon

KeyboardInterrupt: 